In [1]:
!python --version
!pip install -q --upgrade pip
!pip install -q pika 'ultralytics' onnx

Python 3.8.6


In [2]:
CONFIDENCE_NEEDED = .4
PATOLIGHT='192.168.0.14'
LABEL_X_DIST = 146
LABEL_Y_DIST = 230
LABEL_DIST_VARIANCE = .20

In [3]:
import urllib
import json
import time
import os

url='http://'+PATOLIGHT+'/api/control'
print(url)

def setup_robot():
    print("Ready")
    command=url+"?led=00003" # White
    res=urllib.request.urlopen(command)
    time.sleep(5)
    command=url+"?alof=1"
    res=urllib.request.urlopen(command)

def motion_hit(sp):
    print("HIT ME!")
    command=url+"?led=00030" # Blue
    res=urllib.request.urlopen(command)
    time.sleep(5)
    command=url+"?alof=1"
    res=urllib.request.urlopen(command)

    
def motion_hold(sp):
    print("I'm good. HOLD!")
    command=url+"?led=00300" # Green
    res=urllib.request.urlopen(command)
    time.sleep(5)
    command=url+"?alof=1"
    res=urllib.request.urlopen(command)
    
    
def motion_bust(sp):
    print("Awe maan!  Let's go to the slots")
    command=url+"?led=10000" # Red
    res=urllib.request.urlopen(command)
    time.sleep(5)
    command=url+"?alof=1"
    res=urllib.request.urlopen(command)
    
def motion_ready(sp):
    print("Let's Play some Blackjack!")
    command=url+"?led=11111" # White
    res=urllib.request.urlopen(command)
    time.sleep(5)
    command=url+"?alof=1"
    res=urllib.request.urlopen(command)

http://192.168.0.14/api/control


In [4]:
def score_hand(hand):
    score = 0
    aceCount = 0
    for card in hand:
        print(card[0])
        if card[0] in {'K','Q','J'}:
            score += 10
        elif card[0] == 'A':
            aceCount+=1
        elif card[0] == '1' and card[1] == '0':
            score += 10
        else:
            score += int(card[0])
    print("Ace count: " + str(aceCount))
    if aceCount == 2 and score == 0:
        #only have 2 aces, count 1 as value 11 and the other as value 1
        score = 12
    elif aceCount == 2 and score > 9:
        #can't count as 12, need to score both as value 1
        score += 2
    elif aceCount == 2 and score <= 9:
        #add 12, one is value 11, one is value 1
        score += 12
    elif aceCount == 1 and score == 0:
        #only have one ace
        score = 11
    elif aceCount == 1 and score > 10:
        #counting as 11 would bust
        score += 1
    elif aceCount == 1 and score <= 10:
        #count as 11
        score += 11
    elif aceCount > 2:
        #applying simple rule here :-)
        score += aceCount

    return score

In [5]:
global myHand
myHand = {}
global prev_card
prev_card = ''

def add_keys_nested_dict(d, keys):
    for key in keys:
        if key not in d:
            d[key] = {}
        d = d[key]
    d.setdefault(keys[-1], 1)
    
def check_opposite_label(x, y):
    for card, details in myHand.items():
        print(details)
        if len(details) > 0:
            if abs(details['x'] - x) <= (LABEL_X_DIST + LABEL_DIST_VARIANCE*LABEL_X_DIST) and abs(details['y'] - y) <= (LABEL_Y_DIST + LABEL_DIST_VARIANCE*LABEL_Y_DIST):
                print("second label from captured card: " + card)
                return card
    return ''

prev_score=0

def card_inference(msg, prev_card):
  global myHand
  global prev_score
  hand_score = 0
  print(" [x] Received " + str(msg))
  cardSeen = json.loads(msg)
  card = cardSeen["class"]

  if float(cardSeen["score"])>CONFIDENCE_NEEDED and card not in myHand.keys():
        if card == "NONE":
            myHand = {}
            motion_ready(80)
        else:
            #check if we have another card recorded at roughly same location, replace if confidence higher
            overlapping_card = check_opposite_label(float(cardSeen['x1']), float(cardSeen['y1']))
            if len(overlapping_card) > 0 and myHand[overlapping_card]['confidence'] < float(cardSeen["score"]):
                del myHand[overlapping_card]
                myHand[card] = {}
                myHand[card]['x'] = float(cardSeen['x1'])
                myHand[card]['y'] = float(cardSeen['y1'])
                myHand[card]['confidence'] = float(cardSeen["score"])
            if len(overlapping_card) == 0:
                myHand[card] = {}
                myHand[card]['x'] = float(cardSeen['x1'])
                myHand[card]['y'] = float(cardSeen['y1'])
                myHand[card]['confidence'] = float(cardSeen["score"])
                
            hand_score = score_hand(myHand)
            
            if hand_score != prev_score:
                prev_score = hand_score
                if hand_score > 21:
                    motion_bust(50)
                elif hand_score > 15:
                    motion_hold(50)
                else: 
                    motion_hit(50)
        print('My Hand:' + str(myHand) + ', score: ' + str(hand_score))
  return card

In [6]:
import pika, os

rabbitmq_hostname = os.environ.get(
    'AMQP_HOSTNAME', 'localhost'
)
rabbitmq_port = os.environ.get(
    'AMQP_PORT', '5672'
)
rabbitmq_user = os.environ.get(
    'AMQP_USER', 'user'
)
rabbitmq_password = os.environ.get(
    'AMQP_PASSWORD', 'guest'
)

print (rabbitmq_user, rabbitmq_password)
credentials = pika.PlainCredentials(rabbitmq_user, rabbitmq_password)
parameters = pika.ConnectionParameters(rabbitmq_hostname,
                                       rabbitmq_port,
                                       '/',
                                       credentials)

connection = pika.BlockingConnection(parameters)
channel = connection.channel() # start a channel

# Declare a Stream, named test_stream
channel.queue_declare(
  queue='inferencing_stream',
      durable=True,
  arguments={"x-queue-type": "stream", "x-max-age": "1m"}
)

default_user_6zhHiTST6cePTcMNvg6 YfrMJSlmL-GoAKxLenbJ4iwOM66YuDl5


<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=11489', 'queue=inferencing_stream'])>"])>

In [7]:
def ack_message(channel, delivery_tag):
    """Note that `channel` must be the same pika channel instance via which
    the message being ACKed was retrieved (AMQP protocol constraint).
    """
    if channel.is_open:
        channel.basic_ack(delivery_tag)

# create a function which is called on incoming messages
def hande_inference_message(ch, method, properties, body):
  #print(body)
  global prev_card
  prev_card = card_inference(body, prev_card)
  ack_message(ch,method.delivery_tag)

# Set the consumer QoS prefetch
channel.basic_qos(
  prefetch_count=100
)

# Consume messages published to the stream
channel.basic_consume(
  'inferencing_stream',
  hande_inference_message,
  arguments={"x-stream-offset": "last"}
)

'ctag1.d95efadc5e744118af62e9037ebf95e5'

In [ ]:
mc = setup_robot()

# start consuming (blocks)
channel.start_consuming()

dealMore=True

connection.close()
print(myHand)

Ready
 [x] Received b'{"class": "NONE", "score": "1", "x1": "0", "y1": "0" }'
Let's Play some Blackjack!
My Hand:{}, score: 0
